# Deep learning - Una introducción a Keras

In [1]:
# declarar una seed para que los resultados sean reproducibles
seed = 2023

# fijar la semilla aleatoria en NumPy y TensorFlow
import numpy as np
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)

2023-10-31 16:19:04.568882: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 16:19:04.986193: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-31 16:19:04.986244: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-31 16:19:05.079842: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ModuleNotFoundError: No module named 'google.protobuf'

## Preparación de datos

In [ ]:
import pandas as pd
import	os

In [ ]:
# cargar el dataset
FOLDER = 'datasets/'
FILE = 'iris.csv'
path = os.path.join(FOLDER, FILE)

In [ ]:
# leer el dataset con pandas
df = pd.read_csv(path)
print(df.head())

In [ ]:
# codificar la variable Species en one-hot
df = pd.get_dummies(df, columns=['Species'])
print(df.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separar características y etiquetas
x = df.drop(['Species_Iris-setosa', 'Species_Iris-versicolor', 'Species_Iris-virginica'], axis=1)
x = x.drop(['Id'], axis=1)
y = df[['Species_Iris-setosa', 'Species_Iris-versicolor', 'Species_Iris-virginica']]

# Separar los datos en entrenamiento, validación y prueba con una proporción de 60%, 20% y 20% respectivamente
# Separar datos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=seed)

# Separar datos de entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=seed)

# Normalizar características
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

print('x_train shape:', x_train.shape)
print('x_val shape:', x_val.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_val shape:', y_val.shape)
print('y_test shape:', y_test.shape)


Lo que he hecho es dividir el conjunto de datos en 3: entrenamiento, validación y prueba. 

**Conjunto de Entrenamiento:** Es el principal conjunto de datos utilizado para entrenar el modelo. Los algoritmos aprenden a reconocer patrones y hacer predicciones basadas en este conjunto.

**Conjunto de Validación o Desarrollo:** Se utiliza para ajustar y optimizar el modelo. Por ejemplo, durante el entrenamiento, puedes usar este conjunto para probar el rendimiento del modelo y ajustar hiperparámetros. Es fundamental para identificar problemas como el sobreajuste.

**Conjunto de Prueba:** Sirve para evaluar el rendimiento del modelo después del entrenamiento y la validación. Proporciona una estimación imparcial de cómo el modelo funcionará en datos no vistos.

**Proporciones de División:**

En la era anterior del aprendizaje profundo, era común dividir los datos en proporciones como 70/30 (entrenamiento/prueba) o 60/20/20 (entrenamiento/validación/prueba). Sin embargo, con la llegada de grandes conjuntos de datos, estas proporciones están cambiando. Por ejemplo, si se cuenta con un millón de ejemplos, es posible que solo el 1% o incluso menos sea usado para validación y prueba, dejando el 98% o más para el entrenamiento.



## Construcción del modelo en Keras:


**1. Clase `Sequential`:**

La clase `Sequential` es una forma lineal y sencilla de definir modelos en Keras. Permite apilar capas de manera secuencial, una encima de la otra, para construir la arquitectura del modelo.

**Métodos principales:**

- **add()**: Añade una capa al modelo.
- **compile()**: Configura el proceso de aprendizaje del modelo.
- **fit()**: Entrena el modelo con los datos.
- **evaluate()**: Evalúa el rendimiento del modelo.
- **predict()**: Realiza predicciones con el modelo entrenado.

**Uso básico:**
```python
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(32, input_shape=(10,)))
```

---

**2. Función `add`:**

La función `add()` pertenece a la clase `Sequential` y se utiliza para agregar una capa a la arquitectura del modelo. Las capas se añaden en el orden en que se invoca esta función. Por ejemplo:

```python
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(10,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
```

En el código anterior, estamos agregando tres capas densamente conectadas. La primera capa tiene 32 unidades, la segunda 16 unidades y la última tiene 1 unidad.

---

**3. Clase `Dense`:**

La clase `Dense` representa una capa densamente conectada, también conocida como capa completamente conectada. Es una de las capas más comúnmente utilizadas en redes neuronales.

**Parámetros principales:**

- **units**: Número de neuronas en la capa. Es un entero positivo.
- **activation**: Función de activación a utilizar. Puede ser una cadena con el nombre de la función (`'relu'`, `'sigmoid'`, `'softmax'`, etc.) o una función personalizada.
- **input_shape**: Dimensiones de los datos de entrada. Solo es necesario en la primera capa del modelo. Por ejemplo, para datos con 10 características, usaríamos `input_shape=(10,)`.
- **kernel_initializer**: Inicializador para los pesos de las conexiones con las neuronas. Por defecto es `'glorot_uniform'`, pero hay muchas otras opciones disponibles.
- **bias_initializer**: Inicializador para los sesgos. Por defecto es `'zeros'`.
- **kernel_regularizer**: Función regularizadora para los pesos. Puede ser útil para prevenir el sobreajuste.
- **bias_regularizer**: Función regularizadora para los sesgos.

**Uso básico:**
```python
Dense(units=32, activation='relu', input_shape=(10,))
```

---

En resumen:

- La clase `Sequential` permite crear modelos de manera lineal, añadiendo capas de una en una.
- La función `add()` se usa para agregar capas al modelo `Sequential`.
- La clase `Dense` se utiliza para crear capas densamente conectadas con diversos parámetros que permiten personalizar la configuración de estas capas.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_shape=(4,), activation='relu'))  # 4 características de entrada
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 clases en salida
print(model.summary())

## La función softmax

La función **Softmax** es una función utilizada en aprendizaje automático y, específicamente, en redes neuronales, para transformar un vector de números reales z en un vector de valores probabilísticos en el rango [0, 1] que suman 1. Esta función es comúnmente utilizada en la capa de salida de un clasificador multinomial, donde se desea predecir una de múltiples clases posibles.

Dada una entrada z que es un vector de números reales (también conocido como logits), la función Softmax S(z) se define como:

$$ S(z)_j = \frac{e^{z_j}}{\sum_{k=1}^{K} e^{z_k}} $$

Donde:
- $ S(z)_j $ es la salida Softmax para la j-ésima componente del vector z.
- K es el número de clases (o longitud del vector z).
- $e$ es la base del logaritmo natural (aproximadamente igual a 2.71828).

**Intuición:**

1. Para cada componente del vector $z$, se toma la función exponencial $e^{z_j}$. Esto asegura que cada componente de la salida está en el rango positivo.

2. A continuación, se divide cada $e^{z_j}$ por la suma de todas las exponenciales del vector. Esto normaliza los valores de manera que sumen exactamente 1. 

El resultado es un vector donde cada componente representa la probabilidad de que la entrada pertenezca a la clase correspondiente. El valor más alto en el vector resultante indica la clase predicha por el modelo.

**Usos comunes:**

- **Clasificación Multiclase**: Softmax es útil cuando se tiene un problema de clasificación con más de dos clases y las clases son mutuamente excluyentes (es decir, una entrada solo puede pertenecer a una clase).

- **Modelos de Lenguaje**: Se usa en modelos de lenguaje basados en redes neuronales para predecir la siguiente palabra en una secuencia, dada una lista de vocabulario.

**Diferencia con el Sigmoide**: Mientras que la función sigmoide es adecuada para clasificación binaria y transforma un valor real en un valor entre 0 y 1 (lo que puede interpretarse como una probabilidad), la función Softmax generaliza este concepto a múltiples clases, produciendo un vector de valores que suman 1 y que pueden interpretarse como probabilidades para cada clase posible.


El conjunto de datos Iris tiene 3 clases, por lo que la función Softmax producirá 3 valores de salida. Vamos a suponer que, después de procesar una flor Iris específica a través de nuestra red neuronal, obtenemos los siguientes logits para las 3 clases:

$$z = [0.5, 2.5, -1.5]$$

Donde:
-  $z_1$ = 0.5  es el logit para la clase "Iris setosa".
-  $z_2$ = 2.5  es el logit para la clase "Iris versicolor".
-  $z_3$ = -1.5  es el logit para la clase "Iris virginica".

Aplicando la función Softmax:

$$ S(z)_j = \frac{e^{z_j}}{\sum_{k=1}^{3} e^{z_k}} $$

1. Calculamos las exponenciales para cada logit:
$$ e^{z_1} = e^{0.5} \approx 1.65 $$
$$ e^{z_2} = e^{2.5} \approx 12.18 $$
$$ e^{z_3} = e^{-1.5} \approx 0.22 $$

2. Calculamos la suma total de estas exponenciales:
$$ \text{Suma} = 1.65 + 12.18 + 0.22 = 14.05 $$

3. Aplicamos Softmax:
$$ S(z_1) = \frac{1.65}{14.05} \approx 0.117 $$
$$ S(z_2) = \frac{12.18}{14.05} \approx 0.867 $$
$$ S(z_3) = \frac{0.22}{14.05} \approx 0.016 $$

El vector resultante después de aplicar Softmax es:

$$ S(z) \approx [0.117, 0.867, 0.016] $$

Esto indica que nuestro modelo estima que hay:
- Un 11.7% de probabilidad de que la flor sea "Iris setosa".
- Un 86.7% de probabilidad de que sea "Iris versicolor".
- Un 1.6% de probabilidad de que sea "Iris virginica".

De acuerdo con estas probabilidades, nuestro modelo clasificaría esta flor como "Iris versicolor".

## Compilación y entrenamiento:

La función `compile()` de un modelo en Keras es fundamental, ya que especifica cómo se entrenará el modelo. Aquí están los parámetros más importantes:

1. **optimizer**: Es el algoritmo de optimización que se utilizará para actualizar los pesos del modelo. Los optimizadores son cruciales ya que determinan cómo se reduce el error durante el entrenamiento. Algunos optimizadores populares incluyen:


   - `'sgd'`: Stochastic Gradient Descent (Descenso de Gradiente Estocástico). Es una versión básica y frecuentemente utilizada del algoritmo de descenso de gradiente. Y es el que habeis visto en teoría, luego explicare que significa estocástico.

   - `'rmsprop'`: Divide la tasa de aprendizaje para un peso por un promedio móvil de las magnitudes de gradientes recientes.

    - `'adam'`: Es una variante del método de descenso de gradiente que computa tasas de aprendizaje adaptativas para cada parámetro. Es uno de los optimizadores más recomendados en muchos escenarios (Este seguramente sea vuestro mejor amigo :D)
   
   También puedes configurar un optimizador con parámetros personalizados:
   ```python
   from keras.optimizers import SGD
   optimizer = SGD(learning_rate=0.001)
   ```

2. **loss**: Es la función de pérdida (o función objetivo) que el modelo intentará minimizar. Depende del tipo de problema que estés tratando:
   
   - Problemas de **regresión**:
     - `'mean_squared_error'` o `'mse'`: Promedio de los cuadrados de las diferencias entre valores reales y predichos.
     - `'mean_absolute_error'` o `'mae'`: Promedio de las diferencias absolutas entre valores reales y predichos.
     
   - Problemas de **clasificación binaria**:
     - `'binary_crossentropy'`: Función de pérdida de entropía cruzada para clasificación binaria.
     
   - Problemas de **clasificación multiclase**:
     - `'categorical_crossentropy'`: Para etiquetas codificadas en one-hot.
     - `'sparse_categorical_crossentropy'`: Para etiquetas como enteros.

     
     
3. **metrics**: Una lista de métricas que se evaluarán por el modelo durante el entrenamiento y la prueba. Por ejemplo, en problemas de clasificación, comúnmente usamos `['accuracy']` para evaluar la precisión del modelo. Estas métricas no se utilizan para entrenar el modelo, sino más bien para evaluar su desempeño.

   Ejemplo:
   ```python
   metrics=['accuracy']
   ```

4. **Nota**: Entraremos mucho más en detalle sobre los optimizadores y funciones de perdida en siguentes seciones, ahora me vale que sepan que existen

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01) # tasa de aprendizaje

model.compile(optimizer=optimizer, loss='MSE', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=150, batch_size=10, validation_data=(x_val, y_val))

# Puedes utilizar el string 'SGD' en lugar de optimizer pero no podrás modificar la tasa de aprendizaje, por defecto es 0.01
# model.compile(optimizer='SGD', loss='MSE', metrics=['accuracy'])

La función `fit()` es uno de los métodos más esenciales de los modelos en Keras. Se utiliza para entrenar el modelo durante un número fijo de épocas (iteraciones en un conjunto de datos).

A continuación, los parámetros más importantes de `model.fit()`:

1. **x**: Datos de entrenamiento. Puede ser un array de NumPy o, en el caso de modelos que tienen múltiples entradas, una lista de arrays de NumPy.

2. **y**: Etiquetas o valores objetivo. Al igual que con `x`, puede ser un array de NumPy o una lista de arrays en el caso de modelos multi-salida.

3. **batch_size**: Número de muestras por actualización del gradiente. Si no se especifica, el valor predeterminado es 32. 

4. **epochs**: Número de épocas para entrenar el modelo. Una época es una iteración sobre todo el conjunto de datos `x` y `y`.

5. **verbose**: Modo detallado. 
   - `0`: Silencioso.
   - `1`: Muestra una barra de progreso.
   - `2`: Muestra una línea por época.

7. **validation_split**: Fracción del conjunto de entrenamiento a utilizar como datos de validación. Por ejemplo, un valor de 0.2 utilizará el 20% de los datos como conjunto de validación y el 80% restante como conjunto de entrenamiento.

8. **validation_data**: Datos específicos de validación. Es una tupla `(x_val, y_val)` en la que `x_val` son los datos de validación y `y_val` son las etiquetas de esos datos. Esta opción omite el uso de `validation_split`.

9. **shuffle**: Si es `True`, baraja las muestras antes de cada época.

12. **initial_epoch**: Época en la que empezar el entrenamiento (útil para reanudar un entrenamiento previo).

---

Un uso típico del método `fit()` para un modelo sería:

```python
history = model.fit(x=train_data, y=train_labels, batch_size=32, epochs=10, validation_split=0.2, verbose=1)
```

En este caso, estamos entrenando el modelo con datos `train_data` y etiquetas `train_labels`, usando un tamaño de batch de 32, durante 10 épocas, con el 20% de los datos usados para validación, y mostrando una barra de progreso.

La salida (asignada a la variable `history` en el ejemplo) es un objeto que registra las métricas de entrenamiento (y validación, si se proporcionan) para cada época. Esto es útil para análisis y visualización después del entrenamiento.

El parámetro `batch_size` es fundamental para entender cómo se entrena un modelo de aprendizaje profundo, así que vamos a desglosarlo:

### **`batch_size` en el contexto del Aprendizaje Profundo**

El `batch_size` se refiere al número de ejemplos de entrenamiento utilizados en una iteración (o paso) para actualizar los pesos del modelo.

### **Entrenamiento por Lotes, Estocástico y Mini-Lote**

La elección del `batch_size` nos lleva a tres modos distintos de entrenamiento:

1. **Entrenamiento por Lotes (Batch Gradient Descent)**:
    - `batch_size` = tamaño del conjunto de entrenamiento (todos los datos)
    - Los pesos se actualizan una vez por época después de haber visto todos los datos.
    - Es determinista, es decir, para los mismos datos de entrada y el mismo modelo inicial, siempre obtendrás el mismo resultado.
    - Puede ser computacionalmente ineficiente para conjuntos de datos grandes, ya que requiere que todo el conjunto de datos esté en memoria.

2. **Entrenamiento Estocástico (Stochastic Gradient Descent, SGD)**:
    - `batch_size` = 1
    - Los pesos se actualizan después de ver cada dato individualmente.
    - Introduce mucho ruido en la actualización de los pesos, lo que puede ayudar a escapar de óptimos locales, pero también puede hacer que el entrenamiento sea más inestable.
    - Generalmente, requiere más épocas para converger en comparación con el entrenamiento por lotes.

3. **Entrenamiento con Mini-Lote (Mini-Batch Gradient Descent)**:
    - `1` < `batch_size` < tamaño del conjunto de entrenamiento
    - Los pesos se actualizan después de ver un subconjunto (mini-lote) de datos.
    - Combina lo mejor de los dos mundos anteriores: es computacionalmente más eficiente que SGD y puede beneficiarse del ruido en las actualizaciones de pesos para escapar de óptimos locales.
    - Es el método más comúnmente utilizado en la práctica.

### **Impacto de `batch_size` en el entrenamiento**

- **Velocidad de entrenamiento**: Un `batch_size` más grande puede procesar los datos más rápidamente, ya que se beneficia de las optimizaciones de hardware, en particular en GPUs. Sin embargo, un `batch_size` demasiado grande puede exceder la memoria del hardware.

- **Convergencia**: Un `batch_size` más pequeño puede introducir suficiente ruido para evitar óptimos locales, pero también puede llevar a una convergencia inestable. Un tamaño de lote más grande proporciona una estimación más precisa del gradiente, pero con menos actualizaciones por época.

- **Calidad del modelo**: No hay un tamaño de lote óptimo que funcione para todos los problemas. Es un hiperparámetro que, en muchos casos, debe ajustarse experimentalmente. En algunos problemas, los modelos entrenados con mini-lotes más pequeños generalizan mejor en datos no vistos, mientras que en otros casos podría ser lo contrario.

### **Consejo práctico**

Cuando no estés seguro de qué `batch_size` usar, los valores comunes que suelen probarse son 32, 64, 128, 256. Sin embargo, siempre depende de la memoria disponible (especialmente si estás utilizando GPUs) y del problema específico que estés abordando. Es útil experimentar y validar el rendimiento del modelo con diferentes tamaños de lote.

## Evaluación

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from matplotlib import pyplot as plt 

plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['accuracy'], label='accuracy')

plt.title('Entrenamiento IRIS')
plt.xlabel('Épocas')
plt.legend(loc="upper left")

plt.show()

In [ ]:
# Print validation accuracy
plt.plot(history.history['val_loss'], label='Loss')
plt.plot(history.history['val_accuracy'], label='accuracy')

plt.title('Validación IRIS')
plt.xlabel('Épocas')
plt.legend(loc="upper left")

plt.show()

## Guardar un modelo en Keras

In [ ]:
FOLDER_TO_SAVE_MODELS = 'models/'
NAME_FILE_IRIS_MODEL = 'iris_model.h5'
path = os.path.join(FOLDER_TO_SAVE_MODELS, NAME_FILE_IRIS_MODEL)

Guardar un modelo entrenado en Keras es sencillo y puede hacerse de varias formas, dependiendo de lo que desees conservar (estructura del modelo, pesos del modelo, configuración de entrenamiento, etc.). Aquí te mostraré dos métodos principales:

Guardar un modelo entrenado en Keras es sencillo y puede hacerse de varias formas, dependiendo de lo que desees conservar (estructura del modelo, pesos del modelo, configuración de entrenamiento, etc.). Aquí te mostraré dos métodos principales:

### 1. Guardar todo el modelo (estructura + pesos + configuración de entrenamiento):

Puedes guardar el modelo completo en un solo archivo. Esto incluirá:
- La arquitectura del modelo
- Los pesos del modelo
- La configuración de entrenamiento (lo que especificaste al compilar el modelo)
- El optimizador y su estado (si deseas reanudar el entrenamiento donde lo dejaste)

Es decir que es un copia identitica del experiemento

In [ ]:
# Guardar el modelo
model.save(path)

# Cargar el modelo
from keras.models import load_model
loaded_model = load_model(path)
print(loaded_model.summary())


### 2. Guardar solo la arquitectura o solo los pesos:

a. **Guardar la arquitectura**:

Puedes guardar la estructura del modelo (sin ningún peso) en formato JSON. Esto es útil sobre todo para temas de documentación o para replicar modelos en otros proyectos.

In [ ]:
# Guardar en formato JSON
json_string = model.to_json()

import json
print(json.dumps(json.loads(json_string), indent=2))

# Para cargar el modelo desde JSON
from keras.models import model_from_json
loaded_model = model_from_json(json_string)

b. **Guardar solo los pesos**:

Puedes guardar solo los pesos del modelo, esto es útil cuando veamos transfer learning.

In [ ]:
# Guardar los pesos del modelo
MODEL_WEIGHTS = 'iris_model_weights.h5'
path_to_save_weigths = os.path.join(FOLDER_TO_SAVE_MODELS, MODEL_WEIGHTS)

model.save_weights(path_to_save_weigths)

# Cargar los pesos en un modelo con la misma arquitectura
model.load_weights(path_to_save_weigths)
print(model.summary())

Estos métodos te permiten guardar y cargar modelos en Keras de manera efectiva. Dependiendo de tus necesidades (por ejemplo, si deseas reutilizar solo la arquitectura del modelo en otro proyecto, o si necesitas guardar todo para reanudar el entrenamiento más tarde), puedes elegir el método que más te convenga.

## Producción
Una vez entrenada y testeada la red, podemos ponerla en producción. Si queremos hacer una clasifiación invocaremos el método predict del modelo.

Vamos a ver qué resultados nos ofrece la red si introducimos el conjunto de test.

In [ ]:
import numpy as np

# pass y_test to a numpy numerical array
y_test = np.array(y_test)

predictions = model.predict(x_test)
for p, l in zip(predictions, y_test):
    print(p, "->", l)
    if np.argmax(p) == np.argmax(l):
        print(p, "->", l)
    else:
        print(p, "->", l, "✘")

## Deep Learning para Regression

Para la regresión lo único que necesitamos es modificar la capa de salida, en este caso, en lugar de una capa con una neurona y una función de activación, usaremos una capa con una neurona y sin función de activación. Esto es porque queremos que la salida sea un valor continuo y no una probabilidad.

In [ ]:
from keras.datasets import boston_housing
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

# Normalizar datos
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Crear modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer='SGD', loss='mse', metrics=['mae'])

# Entrenar modelo
history = model.fit(train_data, train_targets, epochs=80, batch_size=16, validation_split=0.2)


In [ ]:
# Evaluar modelo
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(f'MAE: {test_mae_score}')

In [ ]:
from matplotlib import pyplot as plt 

plt.plot(history.history['mae'], label='MAE')
plt.plot(history.history['val_mae'], label='val MAE')

plt.title('Entrenamiento Boston Housing')
plt.xlabel('Épocas')
plt.legend(loc="upper right")

plt.show()

## Visión por computador

Vamos a crear una red neuronal para tratar ahora con un conjunto mucho más grande, el dataset MNIST. Verás que, a medida que tratamos con conjuntos mayores, el tiempo de procesamiento se incrementa y la necesidad de contar con una GPU crece al mismo ritmo. Échale un vistazo a este [vídeo](https://www.youtube.com/watch?v=qcOjR-sJkUY&ab_channel=CayetanoGuerra) para que te vayas familiarizando con Google Colab, por si necesitas usarlo.

[Aquí](https://www.adictosaltrabajo.com/2019/06/04/google-colab-python-y-machine-learning-en-la-nube/) tienes un buen tutorial sobre Google Colab

### Conjunto MNIST
El [conjunto MNIST](https://en.wikipedia.org/wiki/MNIST_database) está formado por 70.000 imágenes de dígitos manuscritos del 0 al 9 con un tamaño de 28x28 en escala de grises. A su vez, el conjunto se divide en 60.000 imágenes para entrenamiento y 10.000 para test.

Vamos a utilizar este *dataset* para entrenar una red y ver qué precisión obtenemos al clasificar el conjunto de test. Piensa bien lo que pretendemos lograr: **hacer una red que será capaz de “ver”**, aunque, por ahora, solo sean imágenes de dos dimensiones.

El conjunto MNIST es muy popular y se utiliza mucho para aprender y probar redes, así que Keras ya lo incluye como parte de la librería.

In [ ]:
from keras.datasets import mnist
from matplotlib import pyplot as plt
import numpy as np

# Cargar datos
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Veamos la forma tiene x_train
print("Shape:", train_images.shape)  # 60.000 imágenes de 28x28

# Veamos una imagen cualquiera, por ejemplo, con el índice 125
image = np.array(train_images[125], dtype='float')
plt.imshow(image, cmap='gray')
plt.show()

print("Label:", train_labels[125])
print("Class:\n", np.unique(train_labels))

También es necesario saber en qué rango de valores se mueven nuestras muestras. Vemos que cada pixel es un byte con un rango de valores que va desde el 0 hasta el 255 en formato entero. Esta escala no es muy adecuada para la red. Podemos facilitar mucho el trabajo de entrenamiento si transformamos esta escala en otra centrada en el 0 y con un rango de valores entre -0.5 y 0.5. Y, por supuesto, en formato real.


In [ ]:
# Normalizar imágenes
print("Max value:", max(train_images[125].reshape(784)))
print("Min value:", min(train_images[125].reshape(784)))

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical

# Cargar datos
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalizar imágenes
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convertir etiquetas a formato categórico
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Crear modelo
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='SGD',
              loss='MSE',
              metrics=['accuracy'])

# Entrenar modelo
history = model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)

**Flatten**: Esta capa se encarga de convertir la imagen de 28x28 en un vector de 784 elementos. Es decir, aplana la imagen.


In [ ]:
# Evaluar modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', np.round(test_acc, 2))

In [ ]:
print(model.summary())

In [ ]:
from matplotlib import pyplot as plt 

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')

plt.title('Entrenamiento MNIST')
plt.xlabel('Épocas')
plt.legend(loc="lower right")

plt.show()

## Ejercicios

1. Experimenta con los hiperparametros de la red para los tres conjuntos de datos
    - Número de capas
    - Número de neuronas por capa
    - Funciones de activación
    - Optimizador
    - Función de pérdida
    - Número de épocas
    - Tamaño del batch
    - etc.
2. Anota los resultados obtenidos en una tabla, para cada conjunto de datos, con los hiperparametros que mejor resultado te han dado.
3. ¿Qué conjunto de datos es más dificil de clasificar? ¿Por qué?
4. ¿Qué configuraciones dieron el mejor rendimiento y por qué crees que sucedió?


## Referencias

- [Keras](https://keras.io/)
- [Keras Tutorial: Deep Learning in Python](https://www.datacamp.com/community/tutorials/deep-learning-python)
- [Keras Tutorial: How to get started with Keras, Deep Learning, and Python](https://www.pyimagesearch.com/2016/07/18/installing-keras-for-deep-learning/)
- [Redes neuronales 4](https://nbviewer.org/url/cayetanoguerra.github.io/ia/nbpy/redneuronal4.ipynb)
- [Redes neuronales 5](https://nbviewer.org/url/cayetanoguerra.github.io/ia/nbpy/redneuronal5.ipynb)